In [90]:
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np


In [91]:
posts = pd.read_csv('post_img_polarity.csv', encoding='latin-1')
result = pd.read_csv('cf_report_1272911_aggregated.csv', encoding='latin-1')

In [92]:
#Ajustando as classes do ground truth

result_clean = result[result['qual_o_sentimento_predominante_no_texto'] >= 0]
result_clean = result_clean[result_clean['qual_o_sentimento_predominante_no_texto'] <= 2]
result_clean = result_clean[result_clean['what_is_the_sentiment_feeling_of_the_image'] != -1]
result_clean = result_clean[result_clean['what_is_the_sentiment_feeling_of_the_image'] != 1]

result = result_clean

In [93]:
#Ajustando as classes das predições

posts['img_polarity'] = posts['img_polarity'].values.astype(int)
posts = posts[posts['img_polarity'] >= 0]

posts['polarity'] = posts['polarity'].values.astype(int)
posts = posts[posts['polarity'] >= 0]

def polarity(post):
    if post['polarity'] > 5:
        return 2 #positive
    elif post['polarity'] < -0:
        return 0 #negative
    else:
        return 1 #neutral

posts['post_polarity'] = posts.apply(polarity, axis=1)

In [94]:
#Funções para merge dos datagramas

def alterNumber(post):
    post = post.replace(',','.')
    post = float(post)
    post = int(post)
    
    return post


def remove(val):
    if type(val) != float:
        return val[7:]
    else:
        return val
    
    
result['img']=result['img'].apply(remove)

posts.rename(columns={"blog.id": "blogid"}, inplace=True)

posts['blogid']=posts['blogid'].apply(alterNumber)
result['blogid']=result['blogid'].apply(alterNumber)

resultado = pd.merge(posts, result, on='img')

In [95]:
# Classification_Report

label_img_orig = resultado['what_is_the_sentiment_feeling_of_the_image']
label_img_pred = resultado['img_max_polarity_idx']

print(classification_report(label_img_orig, label_img_pred))

             precision    recall  f1-score   support

        0.0       0.42      0.64      0.51        70
        2.0       0.84      0.69      0.76       198

avg / total       0.73      0.68      0.69       268



In [96]:
label_txt_orig = resultado['qual_o_sentimento_predominante_no_texto']
label_txt_pred = resultado['post_polarity']

print(classification_report(label_txt_orig, label_txt_pred))

             precision    recall  f1-score   support

          0       0.00      0.00      0.00        69
          1       0.19      0.45      0.27        56
          2       0.57      0.55      0.56       143

avg / total       0.34      0.38      0.35       268



C:\Users\greic\AppData\Roaming\Python\Python36\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [98]:
#discordância aplicada em confusion matrix

from sklearn.metrics import confusion_matrix

print(confusion_matrix(label_img_orig,label_img_pred)) #linha, coluna

[[ 45  25]
 [ 62 136]]


In [99]:
print(confusion_matrix(label_txt_orig,label_txt_pred)) #linha, coluna

[[ 0 40 29]
 [ 0 25 31]
 [ 0 65 78]]


In [100]:
print(confusion_matrix(label_txt_orig,label_img_pred)) #linha, coluna

[[41  0 28]
 [20  0 36]
 [46  0 97]]


In [101]:
print(confusion_matrix(label_txt_orig,label_img_orig)) #linha, coluna

[[ 44   0  25]
 [  9   0  47]
 [ 17   0 126]]


In [103]:
print(confusion_matrix(label_txt_pred,label_img_pred)) #linha, coluna

[[ 0  0  0]
 [60  0 70]
 [47  0 91]]
